In [64]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

In [65]:
# Parameters for Indeed scraper: 
jobTitle = '"Web Developer"' # Use double quotes inside of single quotes for exact match on the title
location = 'United States'
maxJobs = 4100 # Can only grab jobs in increments of 10 for now.

# Target URL Format: https://www.indeed.com/jobs?q=Data+Scientist&l=New+York

def makeURLs(jobTitle, location, maxJobs):
    
    numPages = maxJobs / 10
    numPages = int(numPages)
    
    root = 'http://www.indeed.com/jobs?q='
    
    jobTitle = jobTitle.replace(' ', '+')
    location = location.replace(' ', '+')
    
    URLs = []
    
    pageNum = 0
    for page in range(numPages):
    
        if pageNum == 0:
            seedURL = root + jobTitle + '&l=' + location
        else:
            jobNum = pageNum*10
            jobNum = str(jobNum)
            seedURL = root + jobTitle + '&l=' + location + '&start=' + jobNum
        pageNum = pageNum + 1
        URLs.append(seedURL)
        
    return URLs

seedURLs = makeURLs(jobTitle, location, maxJobs)

In [66]:
# Get URL for every job listed on the page. 

listingURLs = []
    
for seedURL in seedURLs:
    print(seedURL)

    # Get HTML for each Seed URL to extract links to job listing pages.
    page = requests.get(seedURL)
    time.sleep(1) # One second pause between requests.
    soup = BeautifulSoup(page.text, 'html.parser')

    # Loop through html and append to list all page URLs linked to by job title.
    for div in soup.find_all(name='div', attrs={'class':'row'}):

        # Get Listing URL
        for a in div.find_all(name='a', attrs={'data-tn-element':'jobTitle'}):
            listingURLs.append(a['href'])

#Remove URLs containing the string '/pagead/' or '/company'      
listingURLs = [ x for x in listingURLs if '/pagead/' not in x ]
listingURLs = [ x for x in listingURLs if '/company/' not in x ]

http://www.indeed.com/jobs?q="Web+Developer"&l=United+States
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=10
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=20
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=30
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=40
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=50
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=60
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=70
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=80
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=90
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=100
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=110
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=120
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=130
http://www.indeed.com/job

http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=1160
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=1170
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=1180
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=1190
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=1200
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=1210
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=1220
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=1230
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=1240
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=1250
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=1260
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=1270
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=1280
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&sta

http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=2300
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=2310
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=2320
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=2330
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=2340
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=2350
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=2360
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=2370
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=2380
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=2390
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=2400
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=2410
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=2420
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&sta

http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=3440
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=3450
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=3460
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=3470
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=3480
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=3490
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=3500
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=3510
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=3520
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=3530
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=3540
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=3550
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&start=3560
http://www.indeed.com/jobs?q="Web+Developer"&l=United+States&sta

In [67]:
# DataFrame to hold job URLs 
df = pd.DataFrame()
# Set column max_colwidth to compare non-truncated URLs
pd.set_option('display.max_colwidth', 100)

full_urls = []

for suffix in listingURLs:
    full_urls.append('http://www.indeed.com' + suffix)
    
finalURLs = pd.Series(full_urls)
df['url'] = finalURLs.values
df.shape

(3142, 1)

In [68]:
df.to_csv('WD_US_large_urls.csv')

In [69]:
# Drop listings with the same listing_url
# New Dataframe for scraped data
df1 = df.drop_duplicates(subset='url', keep='first')

In [70]:
# Reindex the dataframe
df1 = df1.reset_index(drop=True)
df1.shape

(541, 1)

In [71]:
# Get info from listing and save to lists.
# locations list will likely be slightly longer. 
# We can match to URL and then drop where location is blank.
# Then we can safely add on the other columns.

job_titles = []
companies = []
locations = []
summaries = []

for url in df1['url']:

    # Get HTML of the Page
    time.sleep(1)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
           
    # Get Job Title
    for b in soup.find_all(name='b', attrs={'class': 'jobtitle'}):
        job_titles.append(b.text)
    
    # Get Company Name
    for div in soup.find_all(name='div', attrs={'data-tn-component':'jobHeader'}):
        for span in div.find_all(name='span', attrs={'class': 'company'}):
            companies.append(span.text)

    # Get Location from input
    try:
        location = soup.find('input', {'id': 'where'}).get('value')
        locations.append(location)
    except:
        locations.append('')

    # Get Summary
    for span in soup.find_all(name='span', attrs={'class': 'summary'}):
        summaries.append(span.text)   

In [72]:
# Add location to the dataframe 
locations_series = pd.Series(locations)
df1['location'] = locations_series.values
df1.shape

(541, 2)

In [73]:
#Fill empty strings with 'NaN'
df1['location'].replace('', np.nan, inplace=True)

# Now drop rows with null values
df1.dropna(subset=['location'], inplace=True)

df1.shape

(525, 2)

In [74]:
print(len(job_titles))
print(len(companies))
print(len(summaries))

525
525
525


In [75]:
# Lists -> Series to prepare for DataFrame
job_titles_series = pd.Series(job_titles)
companies_series = pd.Series(companies)
summaries_series = pd.Series(summaries)

In [76]:
# Assign column titles and set values
df1['job_title'] = job_titles_series.values
df1['company'] = companies_series.values
df1['summary'] = summaries_series.values

df1.head()

,url,location,job_title,company,summary
0,http://www.indeed.com/rc/clk?jk=1e4fee953abe2f87&fccid=ee9af136518294bf&vjs=3,Connecticut,Full Stack Web Developer,TRAINOR Associates,We’re always looking for a full stack web developer who writes beautiful code. As a Trainor full...
1,http://www.indeed.com/rc/clk?jk=a4e996f7c7c37381&fccid=6ec8a0e09c50363b&vjs=3,Work at Home,Front End Web Developer,My Digital Shield,Work from home or our office\n
2,http://www.indeed.com/rc/clk?jk=10fe55af918df0d0&fccid=bb307ecf9644ad08&vjs=3,"Palo Alto, CA",Web Developer - CSS/HTML,Phantom,"We are looking for an outstanding Web Developer to be responsible for the coding, innovative des..."
3,http://www.indeed.com/rc/clk?jk=f503e11dd33eb42b&fccid=9be1e169d2e16d28&vjs=3,Remote,Front-end Web Developer,Modern Message,At Modern Message our focus is on building great products that solve real-world problems for our...
4,http://www.indeed.com/rc/clk?jk=59875f2da79a0b14&fccid=38be0499a8c344c0&vjs=3,Remote,Web Developer,Steady Vision,We are digital marketing agency seeking a web developer who exhibits strong attention to detail ...


In [77]:
# Export to CSV
df1.to_csv('WD_US_large.csv')